In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [147]:
#Requirements for the Depth estimator
#!conda install pytorch=0.4.1 torchvision=0.2.1 -c pytorch
%pip install tensorboardX==1.4
#!conda install opencv=3.3.1
%pip install networks

import os
path_estimator='/content/drive/My Drive/Colab Notebooks/DepthEstimator' #DNN model 
os.chdir(path_estimator)
os.getcwd()

if 'monodepth2' in os.listdir():
  print('Already cloned')
else:
    !git clone https://github.com/nianticlabs/monodepth2

Already cloned


In [10]:
import matplotlib.pyplot as plt
path_data='/content/drive/My Drive/Colab Notebooks/yolov5/data/videos/video_cedric_frames/' #Original unmodified data

#Some usefull functions
def get_file_list_completed(path_people,extension=".jpg"):
  filenames=[]
  width={}
  height={}
  for item in os.listdir(path_people):
    if item[-4:]==extension:
      filenames.append(item)
  img=plt.imread(path_data+filenames[0])
  width=img.data.shape[1]
  height=img.data.shape[0]
  return filenames,width,height
filenames,width,height=get_file_list_completed(path_data)
print(len(filenames))

1322


In [ ]:
#WARNING : RUN ONLY ONCE
#Create repertory to store the results (and empty it if it is not empty)
import shutil
path_depth='/content/drive/My Drive/Colab Notebooks/yolov5/runs/detect/video_cedric_people3/depth' #Place to store the results
try:
  shutil.rmtree(path_depth) #Delete all previous work if exists
except:
  pass
os.mkdir(path_depth)
os.mkdir(path_depth+'/npy')
os.mkdir(path_depth+'/jpeg')
os.mkdir(path_depth+'/merge')
os.chdir(path_estimator)

In [ ]:
#ATTENTION CHANGER MANUELLEMENT LES REPERTOIRES!
#RUN ONLY ONCE : BUILD THE DEPTH MAPS
!python monodepth2/test_simple.py --image_path '/content/drive/My Drive/Colab Notebooks/yolov5/data/videos/video_cedric_frames/' --model_name mono+stereo_640x192  

-> Loading model from  models/mono+stereo_640x192
   Loading pretrained encoder
   Loading pretrained decoder
-> Predicting on 1322 test images
   Processed 1 of 1322 images - saved prediction to /content/drive/My Drive/Colab Notebooks/yolov5/data/videos/video_cedric_frames/frame_1316_disp.jpeg
   Processed 2 of 1322 images - saved prediction to /content/drive/My Drive/Colab Notebooks/yolov5/data/videos/video_cedric_frames/frame_0324_disp.jpeg
   Processed 3 of 1322 images - saved prediction to /content/drive/My Drive/Colab Notebooks/yolov5/data/videos/video_cedric_frames/frame_0322_disp.jpeg
   Processed 4 of 1322 images - saved prediction to /content/drive/My Drive/Colab Notebooks/yolov5/data/videos/video_cedric_frames/frame_0325_disp.jpeg
   Processed 5 of 1322 images - saved prediction to /content/drive/My Drive/Colab Notebooks/yolov5/data/videos/video_cedric_frames/frame_0323_disp.jpeg
   Processed 6 of 1322 images - saved prediction to /content/drive/My Drive/Colab Notebooks/yolo

In [ ]:
#JUST TO MOVE THE IMAGES AND NPY TO A MORE SUITABLE LOCATION
for file in filenames:
  os.rename(path_data+file[:-4]+'_disp.jpeg',path_depth+'/jpeg/'+file[:-4]+'_disp.jpeg')
  os.rename(path_data+file[:-4]+'_disp.npy',path_depth+'/npy/'+file[:-4]+'_disp.npy')

Now, for each image in path_data, with have a .JPEG (for vizualisation) and a .NPY with a estimation of the depth in path_depth

Le code que nous utilisons n'est pas commenté et j'ai donc dû passer pas mal de temps à vérifier mes intuitions sur ce que cela sortait. L'output est une image de vizualisation mais surtout un NPY qui a une taille constante de 192*640 correspondant à celle du réseau. Le (0,0) est le coin en haut à gauche. Le 192, c'est la hauteur et le 640 c'est la largeur. Les valeurs dans le tableau c'est l'inversion de la distance à l'écran (en tout cas c'est ce qui est dans le code et c'est cohérent avec mes nombreux tests).

In [28]:
#Now, really our code
# Starting with some usefull function again
import numpy as np
def load_depth(file):
  return np.load(path_depth+'/npy/'+file[:-4]+'_disp.npy').reshape((192,640))

def get_depth(file,liste): 
  # Param : file = same format as filename[i] ie JPG image
  # Liste : list of (x,y) points for which we want a depth. 
  # We will load the depth map once and reply for all points
  # For each (x,y) in the list, x is the width (between 0 and 1) and y is the height (between 0 and 1) (0,0) is the corner bottom left
  # WARNING : (COLUMN,ROW) and not the opposite
  if liste==[]: return []
  deep_map=load_depth(file)
  reply=[]
  for (id,x,y) in liste:
    xbis=int(x*640/width)
    ybis=int(y*192/height)
    if xbis<0: xbis=0
    if xbis>=640: xbis=639
    if ybis<0: ybis=0
    if ybis>=192: ybis=191
    deepth=1/deep_map[ybis,xbis]
    reply.append(deepth)
    #print(id,x,xbis,y,ybis,deepth)
  return reply

def load_label_complete(file):
  try:
    label_file=open(path_label_complete+file[:-4]+'.txt',"r")
    lines=label_file.readlines()
    label_file.close
    data=[]
    for line in lines:
      val=np.array(line.split())[0:11]  #
      data.append(val)
    data=np.array(data)
    return data
  except:
    return np.zeros((0,11))

def get_list_from_labels(label): #Renvoi le centre des masques (ou des peoples si pas de masque détecté)
  liste=[]
  for item in label:
    #x=(float(item[7])+float(item[9]))/2
    #y=(float(item[8])+float(item[10]))/2
    x=(float(item[1])+float(item[3]))/2
    y=(float(item[2])+float(item[4]))/2
    id=int(float(item[0]))
    liste.append([id,x,y])
  return liste

def get_3Dcoord(file,f = 1.5,multi=5,x_center=0.5,y_center=0.5):
  coord3D=[]
  label=load_label_complete(file)
  liste=get_list_from_labels(label)
  depth=get_depth(file,liste)
  for i in range(len(liste)):
    id,x,y=liste[i]
    d=depth[i]*multi
    x0 = (x/width - x_center) * d * f
    y0 = (y/height - y_center) * d * f /width*height
    coord3D.append(np.asarray([id,x0,y0,d]))
  return coord3D

def get_coord(id,coord3D):
  for (i,x,y,z) in coord3D:
    if (int(i)==id):
      return (True,x,y,z)
  return (False,0,0,0)

In [6]:
#Emplacement du résultat du tracker
path_depth='/content/drive/My Drive/Colab Notebooks/yolov5/runs/detect/video_cedric_people3/depth' #Place to store the results
path_with_tracker='/content/drive/My Drive/Colab Notebooks/yolov5/runs/detect/video_cedric_people3/labels_complete/results_eulalie/frames'
path_without_tracker='/content/drive/My Drive/Colab Notebooks/yolov5/runs/detect/video_cedric_people3/labels_complete/results'
path_label_complete='/content/drive/My Drive/Colab Notebooks/yolov5/runs/detect/video_cedric_people3/labels_complete/results_eulalie/'

In [110]:
#Code to put 4 images together
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

def merge_and_write_depth_image(input_image,texte):
  img1=plt.imread(os.path.join(path_data+"/"+input_image))
  height1,width1,_1=img1.data.shape
  img2=plt.imread(os.path.join(path_with_tracker+"/"+input_image))
  img3=plt.imread(os.path.join(path_without_tracker+"/"+input_image))
  img3b = cv2.resize(img3, dsize=(width1, height1), interpolation=cv2.INTER_CUBIC)
  img4=plt.imread(os.path.join(path_depth+"/jpeg/"+input_image[:-4]+'_disp.jpeg'))

  height2,width2,_2=img2.data.shape
  height3,width3,_3=img3b.data.shape
  height4,width4,_4=img4.data.shape
  #print(height1,height2,height3,height4)

  assert (width1==width2==width3==width4 and height1==height2==height3==height4  and _1==_2==_3==_4==3)
  img=np.ndarray(shape=(height1*2,width1*2,3))
  img[:height1,:width1,:]=img1
  img[height1:,:width1,:]=img3b
  img[:height1,width1:,:]=img2
  img[height1:,width1:,:]=img4
  num_ligne=0
  for item in texte:
    cv2.putText(img, item, (width1+150, height1+50+num_ligne*35), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3)
    num_ligne=num_ligne+1
  img = img[:, :, [2, 1, 0]]
  cv2.imwrite(path_depth+'/merge/'+input_image, img)


In [137]:
#Euclidian distance
def distance(l1,l2):
  x1,y1,z1=l1
  x2,y2,z2=l2
  return ((x1-x2)**2+(y1-y2)**2+(z1-z2)**2)**(0.5)

#Compute the distance at frame i and a smooth version of it
d={}
d2={}
for (perso1,perso2) in compare:
  d[(perso1,perso2)]={}
  d2[(perso1,perso2)]={}
for (perso1,perso2) in compare:
  for i in range(len(filenames)):
    if i in position[perso1] and i in position[perso2]:
      d[(perso1,perso2)][i]=distance(position[perso1][i],position[perso2][i])
  for i in range(len(filenames)):
    if i in position[perso1] and i in position[perso2]:
      total=0
      count=0
      for j in range(i-10,i+10):
        if j in d[(perso1,perso2)]:
          total = total + d[(perso1,perso2)][j]
          count = count+1
      d2[(perso1,perso2)][i]=total / count

#Build the text to be outputed
def get_text(i):
  texte=[]
  for id in ids:
    if i in position[id]:
      x,y,z=position[id][i]
      texte.append("Coord de "+str(id)+" : "+str(x)[0:5]+" "+str(y)[0:5]+" "+str(z)[0:5])
  texte.append(" ")
  for (perso1,perso2) in compare:
    if i in d2[(perso1,perso2)]:
      texte.append("Distance entre "+str(perso1)+" et "+str(perso2)+" = "+str(d2[(perso1,perso2)][i])[0:4]+"m")
  return texte

In [121]:
#NOW IT IS VERY SPECIFIC TO EACH VIDEO

ids=[50,109,110,133,151] # Les personnes que je tracke
compare=[[50,109],[50,110],[133,151]] # The comparison we want to maje
filenames=sorted(filenames)
position={}
for id in ids:
  position[id]={}
for i in range(len(filenames)):
  #print(filenames[i])
  coord3Dliste=get_3Dcoord(filenames[i],f=3.4/5.54,multi=5.54,x_center=0.475,y_center=0.4037) #SPECIFIC PARAM, depending of the camera param
  for id in ids:
    ok,x,y,z = get_coord(id,coord3Dliste) # Param empirique de l'appareil photo
    if ok:
      position[id][i]=(x,y,z)

In [145]:
#GO ! BUILD THE IMAGES
for i in range(len(filenames)):
  texte=get_text(i)
  merge_and_write_depth_image(filenames[i],texte)
  print(filenames[i]+" done.")

frame_0000.jpg done.
frame_0001.jpg done.
frame_0002.jpg done.
frame_0003.jpg done.
frame_0004.jpg done.
frame_0005.jpg done.
frame_0006.jpg done.
frame_0007.jpg done.
frame_0008.jpg done.
frame_0009.jpg done.
frame_0010.jpg done.
frame_0011.jpg done.
frame_0012.jpg done.
frame_0013.jpg done.
frame_0014.jpg done.
frame_0015.jpg done.
frame_0016.jpg done.
frame_0017.jpg done.
frame_0018.jpg done.
frame_0019.jpg done.
frame_0020.jpg done.
frame_0021.jpg done.
frame_0022.jpg done.
frame_0023.jpg done.
frame_0024.jpg done.
frame_0025.jpg done.
frame_0026.jpg done.
frame_0027.jpg done.
frame_0028.jpg done.
frame_0029.jpg done.
frame_0030.jpg done.
frame_0031.jpg done.
frame_0032.jpg done.
frame_0033.jpg done.
frame_0034.jpg done.
frame_0035.jpg done.
frame_0036.jpg done.
frame_0037.jpg done.
frame_0038.jpg done.
frame_0039.jpg done.
frame_0040.jpg done.
frame_0041.jpg done.
frame_0042.jpg done.
frame_0043.jpg done.
frame_0044.jpg done.
frame_0045.jpg done.
frame_0046.jpg done.
frame_0047.jp

In [146]:
#Video finale
## SAVE VIDEO 
final_path=path_depth+'/merge'

w, h = cv2.imread(final_path+'/'+filenames[0]).shape[0:2]
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(final_path+'/final.mp4',fourcc, 20.0, (h,w))

for filename in filenames:
  frame = cv2.imread(final_path+'/'+filename)
  out.write(frame)

out.release()
cv2.destroyAllWindows()

In [148]:
#FINISH !
#Below is just code I keep because it was usefull to tune the camera param

Ce qui suit m'a servi à calibrer multi en prenant les bons repères dans l'image et en estimant à 5m la largeur de la rue - Je garde ce code pour mémoire

In [37]:
liste=[[1,398,245],[1,301,289],[3,13,423],[4,456,218],[5,530,257],[6,678,257],[7,791,390]]
depth1=get_depth(filenames[5],liste)
depth2=get_depth(filenames[50],liste)


In [55]:
  x_center=0.475
  y_center=0.4037
  multi=1
  f=3.4
  for i in range(len(liste)):
    id,x,y=liste[i]
    d=depth2[i]*multi
    x0 = (x/width - x_center) * d *f
    y0 = (y/height - y_center) * d * f /width*height
    print(x0,y0,d)

-1.3963049714101743 0.6500521868623719 6.797427852706736
-2.123868818901602 0.9728963476106554 3.8689071463862206
-2.105496992368398 0.9743365562742807 1.341971260572077
0.0 6.758281192426976e-05 9.541102859834405
2.8664594043812874 1.5107790498605251 10.937205994459594
2.8163153671903864 0.49478347725747474 3.5819591315616988
3.137372507669215 1.6108487942212546 2.644317477930155


In [54]:
  for i in range(len(liste)):
    id,x,y=liste[i]
    d=depth2[i]*multi
    x0 = (x/width - x_center) * d * f
    y0 = (y/height - y_center) * d * f /width*height
    print(x0,y0,d)

-1.3963049714101743 0.6500521868623719 23.111254699202902
-2.123868818901602 0.9728963476106554 13.154284297713149
-2.105496992368398 0.9743365562742808 4.562702285945062
0.0 6.758281192426978e-05 32.439749723436975
2.8664594043812874 1.5107790498605251 37.18650038116262
2.8163153671903864 0.49478347725747474 12.178661047309776
3.1373725076692156 1.6108487942212548 8.990679424962527


Et ceci sert à calibrer les paramètres de distances 

In [80]:
distance={}
for i in range(303,579):
  pos1=position[50][i]
  pos2=position[75][i]
  distance[i]=((pos1[0]-pos2[0])**2+(pos1[1]-pos2[1])**2+(pos1[2]-pos2[2])**2)**(0.5)

In [82]:
debut=[306,336,366,396,426,456,486,516,546]
DD=[]
for item in debut:
  dist=0
  for i in range(item, item+30):
    dist=dist+distance[i]
  dist=dist/30
  DD.append(dist)
  print(dist)
for i in range(len(DD)-1):
  print("Vitesse : ",(DD[i+1]-DD[i])*3.6)

3.121023273716644
2.9580802345490866
3.8625273513531084
5.566395294479477
6.199660994224376
7.927923800428172
11.832229730915033
12.9419537604647
11.939372812577604
Vitesse :  -0.586594941003207
Vitesse :  3.2560096204944786
Vitesse :  6.1339245952549275
Vitesse :  2.2797565190816367
Vitesse :  6.221746102333663
Vitesse :  14.055501349752703
Vitesse :  3.9950065063788034
Vitesse :  -3.609291412393549


In [88]:
distance={}
distance2={}
for i in range(727,847):
  pos1=position[102][i]
  pos2=position[107][i]
  pos3=position[109][i]
  distance[i]=((pos1[0]-pos2[0])**2+(pos1[1]-pos2[1])**2+(pos1[2]-pos2[2])**2)**(0.5)
  distance2[i]=((pos1[0]-pos3[0])**2+(pos1[1]-pos3[1])**2+(pos1[2]-pos3[2])**2)**(0.5)

In [90]:
debut=[727,757,777,807]
DD=[]
for item in debut:
  dist=0
  for i in range(item, item+30):
    dist=dist+distance2[i]
  dist=dist/30
  DD.append(dist)
  print(dist)
for i in range(len(DD)-1):
  print("Vitesse : ",(DD[i+1]-DD[i])*3.6)

3.4627041517867285
2.4104740986469557
3.989847347943539
8.700941322184905
Vitesse :  -3.7880281913031824
Vitesse :  5.6857436974677
Vitesse :  16.959938307268917


In [86]:
position[109]

{507: (0.7395013175812334, 1.8513015433348146, 40.42132574708647),
 508: (0.7438924766011669, 1.8387985415914976, 39.91400026882517),
 509: (0.7418935592781437, 1.8290801337717417, 39.64773409089043),
 510: (0.733519796309601, 1.831006697613018, 39.68471808053012),
 511: (0.7375876556232689, 1.8595094263105953, 40.117529476609704),
 512: (0.7441505051424268, 1.8835752222732165, 40.578421205033045),
 513: (0.7427794040661752, 1.9492836078619071, 42.57650480484749),
 514: (0.7231567314123766, 1.9008298900416807, 41.737643143999954),
 515: (0.7217281702617057, 1.8803170235317792, 41.76313436792589),
 516: (0.713306893995097, 1.8433543805334653, 41.31537169021665),
 517: (0.7100125104014269, 1.7984959398266542, 40.63972173970836),
 518: (0.7404048533433872, 1.8526936356152162, 42.19222354042659),
 519: (0.7459267871448175, 1.849224995874877, 42.43780137544447),
 520: (0.741285953122136, 1.8537241071155308, 42.66005033609799),
 521: (0.7368526852947584, 1.8490283607968436, 42.59161067070285